L/S Cell

In [10]:
# Import the needed extensions
import math
import klayout.db as db

def ls_cell(tone:str="D",size:float=500,pitch:float=1500,dimension:float=25000,angle:float=30,x2y:float=1,metro_structure:bool = True):

    #Create the layout
    layout = db.Layout()

    #Convert from nm to um for size and pitch
    size = size/1000
    pitch = pitch/1000

    #Name the topcell
    Unit_main = layout.create_cell(f"LS_Array_{tone}_{size}_{pitch}_{angle}")

    #Defines initial layers
    l_outline = layout.layer(11,0) #overlay layer for the cell
    l_line = layout.layer(2,0) #line layer, sacrificial
    check_layer = layout.layer(98,0) #layer for sliver check, sacrificial
    check_operation = layout.layer(99,0) #layer for sliver check operation, sacrificial

    #Create the overlay shape
    Unit_main.shapes(l_outline).insert(db.DBox((-dimension/2),-dimension/2,(dimension/2),dimension/2))

    #Create check layer shape
    check_factor = 0.99
    check_shape = db.DBox(check_factor*(-dimension/2),check_factor*-dimension/2,check_factor*(dimension/2),check_factor*dimension/2)
    Unit_main.shapes(check_layer).insert(check_shape)

    #Check pitch of the LS array
    pitch_check = math.floor(dimension/(pitch))
    iso = pitch_check < 2 #boolean
    bar3 = (pitch-size) < size/2 #also boolean

    #Define line dimensions
    l_left = -size/2
    l_bottom = -dimension/2
    l_right = size/2
    l_top = dimension/2
    l_2bottom = -dimension
    l_2top = dimension

    #Create the LS array
    if iso:
        Unit_main.shapes(l_line).insert(db.DBox(l_left, l_2bottom, l_right, l_2top))
    elif bar3:
        Unit_main.shapes(l_line).insert(db.DBox(l_left, l_2bottom, l_right, l_2top))
        Unit_main.shapes(l_line).insert(db.DBox(-dimension,-dimension,3*l_left,dimension))
        Unit_main.shapes(l_line).insert(db.DBox(3*l_right,-dimension,dimension,dimension))
    else:
        for num in range(-pitch_check,pitch_check,1):
            coord_x = num*pitch
            Unit_main.shapes(l_line).insert(db.DBox(coord_x+l_left,l_2bottom,coord_x+l_right,l_2top))

    #Add metro structure if applicable

    metro_spacing = 3 #um

    if metro_structure:
        if iso or bar3:
            Unit_main.shapes(l_line).insert(db.DBox(-(2*size),-(metro_spacing+size/2),(2*size),-(metro_spacing-size/2)))
            Unit_main.shapes(l_line).insert(db.DBox(-(2*size),(metro_spacing+size/2),(2*size),(metro_spacing-size/2)))
        else:
            Unit_main.shapes(l_line).insert(db.DBox(-(1.5*size),-(metro_spacing+size/2),(1.5*size),-(metro_spacing-size/2)))
            Unit_main.shapes(l_line).insert(db.DBox(-(1.5*size),(metro_spacing+size/2),(1.5*size),(metro_spacing-size/2)))

    #Does the angle transformation for the lines for rotations
    t = db.ICplxTrans(1,angle,0,0,0)
    Unit_main.shapes(l_line).transform(t)

    #Removes polygons extending beyond the overlay layer by only including those within the overlay region (& statement)
    outline_check = db.Region(Unit_main.shapes(l_outline))
    shapes_check = db.Region(Unit_main.shapes(l_line))
    r_and = shapes_check & outline_check

    #Performs additional operation to remove any slivers from the edge of the design (in the case of dense array only)
    if not iso:
        if not bar3:
            Unit_main.shapes(check_operation).insert(r_and)
            r_and = Unit_main.begin_shapes_rec_touching(check_operation,check_shape)

    #Flips tone using XOR if tone is C instead of D
    if tone == "C":
        r_and = r_and ^ outline_check

    #Adds new layer with finished cell
    l_diff = layout.layer(1,0)
    Unit_main.shapes(l_diff).insert(r_and)


    #Removes sacrificial layers
    layout.delete_layer(l_line)
    layout.delete_layer(check_layer)
    layout.delete_layer(check_operation)

    #Convert back to nm
    size=size*1000
    pitch = pitch*1000

    if iso:
        pitch = 'iso'
    elif bar3:
        pitch = '3bar'

    #Export GDS
    layout.write(f"LS_test_tone-{tone}_size-{size}nm_pitch-{pitch}_dimension-{dimension}um_angle-{angle}_metro-{metro_structure}.gds")

    return layout

ls_cell()